In [104]:
stub='esp'

In [105]:
#laps_fn='/Users/ajh59/Dropbox/F1data/2017/eventTiming/AUS/P1 - Lap Times'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_q0_timing_qualifyingsessionlaptimes_v01'#'lap_analysis_5'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p3_timing_thirdpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p2_timing_secondpracticesessionlaptimes_v01_1'
laps_fn='/Users/ajh59/Downloads/2018_01_aus_f1_p1_timing_firstpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_02_brn_f1_p1_timing_firstpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_02_brn_f1_p3_timing_thirdpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_02_brn_f1_q0_timing_qualifyingsessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_03_chn_f1_p1_timing_firstpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_04_aze_f1_p1_timing_firstpracticesessionlaptimes_v01'
laps_fn='/Users/ajh59/Downloads/2018_05_esp_f1_q0_timing_qualifyingsessionlaptimes_v01'
#laps_fn='/Users/ajh59/Downloads/2018_03_chn_f1_q0_timing_qualifyingsessionlaptimes_v01'
laps_out_fn='{}18_qualilaptimes.csv'.format(stub)
#laps_out_fn='{}18_p3laptimes.csv'.format(stub)
#laps_out_fn='{}17_P1laptimes.csv'.format(stub)

In [106]:
import lxml.etree
import pandas as pd
import os

def xmlify(fn):
    cmd = 'pdftohtml -xml -nodrm -zoom 1.5 -enc UTF-8 -noframes %s "%s" "%s"' % (
        '',fn+'.pdf', os.path.splitext(fn+'.xml')[0])
    # can't turn off output, so throw away even stderr yeuch
    cmd + " >/dev/null 2>&1"
    os.system(cmd)

def loader(fn):
    fnx=fn+'.xml'
    xmldata = open(fn+'.xml','r').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

def loader2018(fn):
    fnx=fn+'.xml'
    #xmldata = open(fn+'.xml','r').read()
    xmldata = open(fn+'.xml','rb').read()
    root = lxml.etree.fromstring(xmldata)
    pages = list(root)
    return pages, root

In [107]:
loader = loader2018

In [108]:
xmlify(laps_fn)

In [109]:
def patcher(laps_fn):
    !sed -i.'' -E 's/(<text .*>)([0-9]+) <b>(.*)<\/b><\/text>/\1\2<\/text>\1<b>\3<\/b><\/text>/g' "{laps_fn}.xml"


In [110]:
patcher(laps_fn)

In [111]:
pages,root=loader(laps_fn)

In [112]:
def flatten(el):           
    result = [ (el.text or "") ]
    for sel in el:
        result.append(flatten(sel))
        result.append(sel.tail or "")
    return "".join(result)

def laptimes_table(pages):
    rows=[]

    data=[]

    for page in pages:
        scraping=False
        prevrow=0
        newpair=False
        for el in page:
            txt=flatten(el).strip()
            tmp={'text':txt}
            if 'left' in el.attrib and el.attrib['left'] is not None:
                for attr in el.attrib:
                    tmp[attr]=el.attrib[attr]
                #print txt, el.attrib
                rows.append(tmp)

                prevrow=int(el.attrib['top'])
                if scraping:
                    if not newpair:
                        newpair=txt
                    else:
                        data.append( (newpair,txt) )
                        newpair=False

                if not scraping and (txt.startswith('2017 FORMULA 1') or txt.startswith('FORMULA 1')):
                    scraping=True

    return data,pd.DataFrame.from_records(rows)

In [113]:
def varasint(cand, retval=''):
    ''' Function to determine if we can cast something as an int.
        If we can, do so and return it, otherwise return retval, or, if retval=='asis', the original value. '''
    retval = int(cand) if (int(cand) if isinstance(cand, str) and cand.isdigit() else False) or isinstance(cand, int) else retval
    if retval=='asis':
        return cand
    elif retval=='boolean':
        return isinstance(retval, int)
    return retval

def laptimes_table2018(pages):
    rows=[]

    data=[]

    for page in pages:
        scraping=True
        prevrow=0
        newpair=False
        for el in page:
            txt=flatten(el).strip()
            tmp={'text':txt}
            if 'left' in el.attrib and el.attrib['left'] is not None:
                for attr in el.attrib:
                    tmp[attr]=el.attrib[attr]
                #print txt, el.attrib
                rows.append(tmp)

                prevrow=int(el.attrib['top'])
                if scraping:
                    if not newpair:
                        newpair=txt
                    else:
                        data.append( (newpair,txt) )
                        newpair=False

                if not scraping and (txt.startswith('2018 FORMULA 1') or txt.startswith('FORMULA 1')):
                    scraping=True

        #last two rows:
        #- name of GP eg FORMULA 1 2018 ROLEX AUSTRALIAN GRAND PRIX - Melbourne
        #- Sheet (eg Qualifying Session Lap Times)
    data = [d for d in data if d[0].startswith('NO') or varasint( d[0].split(' ')[0], 'boolean')]
    return data,pd.DataFrame.from_records(rows)

In [114]:
laptimes_table = laptimes_table2018

In [115]:
data,df=laptimes_table2018(pages)

In [116]:
data,df=laptimes_table(pages)
df[:15]

,font,height,left,src,text,top,width
0,NaN,129,36,/Users/ajh59/Downloads/2018_05_esp_f1_q0_timin...,,33,285
1,NaN,86,693,/Users/ajh59/Downloads/2018_05_esp_f1_q0_timin...,,61,150
2,0,17,35,NaN,2,269,7
3,1,17,71,NaN,Stoffel VANDOORNE,269,106
4,1,17,50,NaN,NO,298,16
5,1,17,113,NaN,TIME,298,27
6,1,17,56,NaN,1,316,15
7,0,17,101,NaN,15:02:59,316,46
8,1,17,56,NaN,2,334,15
9,0,17,101,NaN,1:19.653,334,46


In [117]:
#data

In [118]:
def _grab_laptimes(data):
    name=''
    lap=''
    times=[]
    inlaps=False
    tmp={}
    earliest=None
    for pair in data:
        print(pair)
        if pair==('NO','NO') or pair==('LAP','LAP'):
            if 'laps' in tmp: tmp['laps'].pop()
            number,name=prevpair
            if tmp!={}: times.append(tmp)
            tmp={'name':name,'number':number, 'laps':[]}
        elif pair==('TIME','TIME'):
            inlaps=True
        elif inlaps:
            if earliest is None:
                earliest=(tmp['name'],pair[1])
            elif len(tmp['laps'])==0:
                if earliest[1]>pair[1]:
                    earliest=(tmp['name'],pair[1])
            tmp['laps'].append(pair)

        prevpair=pair
    #Catch tailender
    if tmp!={}: times.append(tmp)
    return earliest,times

def _grab_laptimes2018(data):
    name=''
    lap=''
    times=[]
    inlaps=False
    tmp={}
    earliest=None
    named = False
    for pair in data:
        print(pair)
        if pair==('NO','TIME') and not named:
            if 'laps' in tmp and tmp['laps']: tmp['laps'].pop()
            number,name=prevpair
            if tmp!={}: times.append(tmp)
            tmp={'name':name,'number':number, 'laps':[]}
            named = True
            inlaps=True
        elif pair==('NO','TIME'):
            inlaps=True
            named = False
        elif inlaps:
            if earliest is None:
                earliest=(tmp['name'],pair[1])
            elif len(tmp['laps'])==0:
                if earliest[1]>pair[1]:
                    earliest=(tmp['name'],pair[1])
            tmp['laps'].append(pair)

        prevpair=pair
    #Catch tailender
    if tmp!={}: times.append(tmp)
    return earliest,times

In [119]:
_grab_laptimes = _grab_laptimes2018

In [120]:
earliest,laptimes=_grab_laptimes(data)
earliest

('2', 'Stoffel VANDOORNE')
('NO', 'TIME')
('1', '15:02:59')
('2', '1:19.653')
('3', '1:43.840')
('4', '1:19.562')
('5 P', '1:34.697')
('6', '5:26.248')
('7', '1:19.068')
('8', '1:40.802')
('9', '1:18.885')
('NO', 'TIME')
('10 P', '1:43.422')
('11', '7:25.092')
('12', '1:18.491')
('13', '1:37.561')
('14', '1:18.347')
('15 P', '1:36.947')
('16', '4:52.754')
('17', '1:18.323')
('18 P', '1:48.110')
('3', 'Daniel RICCIARDO')
('NO', 'TIME')
('1', '15:07:24')
('2', '1:18.301')
('3', '1:46.546')
('4', '1:17.623')
('5 P', '1:33.750')
('6', '14:59.367')
('7', '1:17.638')
('8 P', '1:44.124')
('9', '7:03.162')
('NO', 'TIME')
('10', '1:18.936')
('11 P', '1:36.845')
('12', '8:28.751')
('13', '1:16.858')
('14 P', '1:37.143')
('15', '3:24.120')
('16', '1:16.818')
('17', '1:44.347')
('18', '1:16.829')
('5', 'Sebastian VETTEL')
('NO', 'TIME')
('1', '15:02:36')
('2', '1:17.806')
('3', '1:46.557')
('4', '1:17.724')
('5', '1:44.038')
('6', '1:17.031')
('7 P', '1:34.441')
('8', '16:04.134')
('NO', 'TIME')
(

('Lance STROLL', '15:01:47')

In [121]:
names = [ '"{}"=\'\''.format(l['name']) for l in laptimes]
print(', '.join(names))

"Stoffel VANDOORNE"='', "Daniel RICCIARDO"='', "Sebastian VETTEL"='', "Kimi RAIKKONEN"='', "Romain GROSJEAN"='', "Marcus ERICSSON"='', "Pierre GASLY"='', "Sergio PEREZ"='', "Fernando ALONSO"='', "Charles LECLERC"='', "Lance STROLL"='', "Kevin MAGNUSSEN"='', "Nico HULKENBERG"='', "Brendon HARTLEY"='', "Esteban OCON"='', "Max VERSTAPPEN"='', "Sergey SIROTKIN"='', "Lewis HAMILTON"='', "Carlos SAINZ"='', "Valtteri BOTTAS"=''


In [122]:
import datetime
#http://tgs.github.io/nptime/
from nptime import nptime

def nptimify(t):
    tt=t.split(':')
    if len(tt)==3:
        h=int(tt[0])
        m=int(tt[1])
        if len(tt[2].split('.'))==2:
            s=int(tt[2].split('.')[0])
            ms=int(tt[2].split('.')[1])
        else:
            s=int(t.split(':')[2])
            ms=0
    elif len(tt)==2:
        h=int(tt[0])
        m=int(tt[1])
        if len(tt[2].split('.'))==2:
            s=int(tt[2].split('.')[0])
            ms=int(tt[2].split('.')[1])
        else:
            s=int(t.split(':')[2])
            ms=0
    return nptime(h, m, s, ms)

def npreltimify(t,earliest="0:0:0"):
    return nptimify(t) - nptimify(earliest)

def nprebase(delta,earliest="0:0:0"):
    return delta + nptimify(earliest)

#def nprebaseElapsed()

#Preferred time format
def formatTime(t):
    return float("%.3f" % t)
# Accept times in the form of hh:mm:ss.ss or mm:ss.ss
# Return the equivalent number of seconds
def getTime(ts):
    print(ts)
    t=ts.strip()
    t=ts.split(':')
    if len(t)==3:
        tm=60*int(t[0])+60*int(t[1])+float(t[2])
    elif len(t)==2:
        tm=60*int(t[0])+float(t[1])
    else:
        tm=float(t[0])
    return formatTime(tm)

In [123]:
def _get_rebased_laptimes(times,earliest): 
    rr=[]
    for driver in times:
        for lap in driver['laps']:
            if lap[0].split(' ')[0]=='1':
                b=nprebase(npreltimify(lap[1],earliest[1])).isoformat()
            else:
                b=lap[1]
            rr.append({'name':driver['name'],
                       'number':driver['number'], 
                       'lap':lap[0].split(' ')[0], #was lapNumber
                       'pit': True if lap[0].endswith('P') else False,
                       'strtime':b, #was laptime
                       'rawtime':getTime(b) #was stime
                      })
    return pd.DataFrame(rr)

In [124]:
#Patch for italy 2017 where there was a restart
laptimespatch=[]
for lapset in laptimes:
    tmp={'name':lapset['name'],'number':lapset['number'],'laps':[]}
    cnt=0
    started =False
    for (lapnum,laptime) in lapset['laps']:
        if laptime.startswith('16'):
            started =True 
        if started:
            cnt=cnt+1
            tmp['laps'].append( (str(cnt),laptime) )
    laptimespatch.append(tmp)

newearliest=None
for lapset in laptimes:
    if newearliest is None:
        newearliest=(lapset['name'],lapset['laps'][0][1])
    else:
        if len(lapset['laps']) and lapset['laps'][0][1]<newearliest[1]:
            newearliest=(lapset['name'],lapset['laps'][0][1])
newearliest
#earliest=newearliest

#laptimes=laptimespatch

('Lance STROLL', '15:01:47')

In [125]:
lapset['laps'][0][1]

'15:06:10'

In [126]:
for l in laptimes:
    if l['laps']: print(l['laps'][0])

('1', '15:02:59')
('1', '15:07:24')
('1', '15:02:36')
('1', '15:02:27')
('1', '15:02:47')
('1', '15:04:14')
('1', '15:05:08')
('1', '15:03:15')
('1', '15:03:24')
('1', '15:03:18')
('1', '15:01:47')
('1', '15:03:04')
('1', '15:09:11')
('1', '15:02:55')
('1', '15:08:03')
('1', '15:02:19')
('1', '15:05:30')
('1', '15:08:17')
('1', '15:06:10')


In [127]:
laptimes

[{'name': 'Stoffel VANDOORNE',
  'number': '2',
  'laps': [('1', '15:02:59'),
   ('2', '1:19.653'),
   ('3', '1:43.840'),
   ('4', '1:19.562'),
   ('5 P', '1:34.697'),
   ('6', '5:26.248'),
   ('7', '1:19.068'),
   ('8', '1:40.802'),
   ('9', '1:18.885'),
   ('10 P', '1:43.422'),
   ('11', '7:25.092'),
   ('12', '1:18.491'),
   ('13', '1:37.561'),
   ('14', '1:18.347'),
   ('15 P', '1:36.947'),
   ('16', '4:52.754'),
   ('17', '1:18.323'),
   ('18 P', '1:48.110')]},
 {'name': 'Daniel RICCIARDO',
  'number': '3',
  'laps': [('1', '15:07:24'),
   ('2', '1:18.301'),
   ('3', '1:46.546'),
   ('4', '1:17.623'),
   ('5 P', '1:33.750'),
   ('6', '14:59.367'),
   ('7', '1:17.638'),
   ('8 P', '1:44.124'),
   ('9', '7:03.162'),
   ('10', '1:18.936'),
   ('11 P', '1:36.845'),
   ('12', '8:28.751'),
   ('13', '1:16.858'),
   ('14 P', '1:37.143'),
   ('15', '3:24.120'),
   ('16', '1:16.818'),
   ('17', '1:44.347'),
   ('18', '1:16.829')]},
 {'name': 'Sebastian VETTEL',
  'number': '5',
  'laps': [

In [128]:
ddx=_get_rebased_laptimes(laptimes,earliest)
ddx[-5:]

00:01:12
1:19.653
1:43.840
1:19.562
1:34.697
5:26.248
1:19.068
1:40.802
1:18.885
1:43.422
7:25.092
1:18.491
1:37.561
1:18.347
1:36.947
4:52.754
1:18.323
1:48.110
00:05:37
1:18.301
1:46.546
1:17.623
1:33.750
14:59.367
1:17.638
1:44.124
7:03.162
1:18.936
1:36.845
8:28.751
1:16.858
1:37.143
3:24.120
1:16.818
1:44.347
1:16.829
00:00:49
1:17.806
1:46.557
1:17.724
1:44.038
1:17.031
1:34.441
16:04.134
1:16.802
1:41.365
19:42.827
1:17.255
1:33.759
5:53.405
1:16.305
00:00:40
1:17.986
1:39.250
1:17.483
1:59.708
2:25.429
16:26.474
1:17.071
1:37.761
19:39.928
1:18.192
1:26.326
5:11.977
1:16.612
1:54.538
00:01:00
1:19.065
1:39.255
1:18.550
1:36.826
4:02.922
1:18.305
1:44.125
1:18.340
1:44.655
7:47.411
1:18.069
1:34.822
2:05.227
4:04.939
1:17.699
1:42.165
1:45.676
9:12.412
1:18.209
1:36.107
1:18.380
1:31.972
3:27.984
1:17.835
00:02:27
1:19.846
1:47.262
1:42.454
5:03.513
1:19.673
1:51.589
1:19.493
1:46.222
00:03:21
1:19.081
1:40.801
1:18.658
1:37.707
3:22.772
1:37.409
1:36.561
1:18.550
1:48.759
6:32.

,lap,name,number,pit,rawtime,strtime
292,9,Valtteri BOTTAS,77,False,76.909,1:16.909
293,10,Valtteri BOTTAS,77,True,98.686,1:38.686
294,11,Valtteri BOTTAS,77,False,299.433,4:59.433
295,12,Valtteri BOTTAS,77,False,76.213,1:16.213
296,13,Valtteri BOTTAS,77,False,110.707,1:50.707


In [129]:
laps_out_fn

'esp18_qualilaptimes.csv'

In [130]:
!pwd

/Users/ajh59/Dropbox/wranglingf1datawithr/f1djR/scraping


In [131]:
ddx.to_csv(laps_out_fn,index=False)

In [132]:
ddx.head()

,lap,name,number,pit,rawtime,strtime
0,1,Stoffel VANDOORNE,2,False,72.000,00:01:12
1,2,Stoffel VANDOORNE,2,False,79.653,1:19.653
2,3,Stoffel VANDOORNE,2,False,103.840,1:43.840
3,4,Stoffel VANDOORNE,2,False,79.562,1:19.562
4,5,Stoffel VANDOORNE,2,True,94.697,1:34.697


In [133]:
len(ddx['name'].unique())

19

In [134]:
laps_out_fn

'esp18_qualilaptimes.csv'

In [135]:
!pwd


/Users/ajh59/Dropbox/wranglingf1datawithr/f1djR/scraping


In [136]:
host='127.0.0.1'
port=3399
user='root'
passwd='f1'
dbname='ergastdb'

In [137]:
#!/Users/ajh59/anaconda3/bin/pip install pymysql
from sqlalchemy import create_engine

engine = create_engine('mysql+pymysql://{user}:{passwd}@{host}:{port}/{dbname}'.format(host=host,
                                                                                           port=port,
                                                                                           user=user,
                                                                                           passwd=passwd,
                                                                                           dbname=dbname))


In [138]:
name='f1dj_laps_tmp'
ddx.to_sql(name, engine,if_exists='replace')

/Users/ajh59/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:165: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [139]:
ddx.head()

,lap,name,number,pit,rawtime,strtime
0,1,Stoffel VANDOORNE,2,False,72.000,00:01:12
1,2,Stoffel VANDOORNE,2,False,79.653,1:19.653
2,3,Stoffel VANDOORNE,2,False,103.840,1:43.840
3,4,Stoffel VANDOORNE,2,False,79.562,1:19.562
4,5,Stoffel VANDOORNE,2,True,94.697,1:34.697


In [140]:
pd.read_sql('SELECT * FrOM f1dj_laps_tmp LIMIT 10', engine)

,index,lap,name,number,pit,rawtime,strtime
0,0,1,Stoffel VANDOORNE,2,0,72.000,00:01:12
1,1,2,Stoffel VANDOORNE,2,0,79.653,1:19.653
2,2,3,Stoffel VANDOORNE,2,0,103.840,1:43.840
3,3,4,Stoffel VANDOORNE,2,0,79.562,1:19.562
4,4,5,Stoffel VANDOORNE,2,1,94.697,1:34.697
5,5,6,Stoffel VANDOORNE,2,0,326.248,5:26.248
6,6,7,Stoffel VANDOORNE,2,0,79.068,1:19.068
7,7,8,Stoffel VANDOORNE,2,0,100.802,1:40.802
8,8,9,Stoffel VANDOORNE,2,0,78.885,1:18.885
9,9,10,Stoffel VANDOORNE,2,1,103.422,1:43.422
